## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import nltk
import time
import pickle

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix 

In [4]:
pd.set_option('display.max_colwidth', None)

## Task 1: Loading the dataset

In [5]:
chatgpt_train = pd.read_csv("chatgpt_train.csv")
chatgpt_train.head()

,date,title,review,rating
0,5/21/2023 16:42,Much more accessible for blind users than the web version,"Up to this point I?€?ve mostly been using ChatGPT on my windows desktop using Google Chrome. While it?€?s doable, screen reader navigation is pretty difficult on the desktop site and you really have to be an advanced user to find your way through it. I have submitted numerous feedbacks to open AI about this but nothing has changed on that front.\nWell, the good news ?€? the iOS app pretty much addresses all of those problems. The UI seems really clean, uncluttered and designed well to be compatible with voiceover, the screen reader built into iOS. I applaud the inclusivity of this design ?€? I only wish they would give the same attention and care to the accessibility experience of the desktop app.\nI would have given this review five stars but I have just a couple minor quibbles. First, once I submit my prompt, voiceover starts to read aloud ChatGPT?€?s response before that response is finished, so I will hear the first few words of the response followed by voiceover reading aloud the ?€?stop generating?€? button, which isn?€?t super helpful. It would be great if you could better coordinate this alert so that it didn?€?t start reading the message until it had been fully generated. The other thing I?€?d like is a Feedback button easily accessible from within the main screen of the app, to make it as easy as possible to get continuing suggestions and feedback from your users.\nOtherwise, fantastic app so far!",4
1,7/11/2023 12:24,"Much anticipated, wasn?€?t let down.","I?€?ve been a user since it?€?s initial roll out and have been waiting for a mobile application ever since using the web app. For reference I?€?m a software engineering student while working in IT full time. I have to say GPT is an crucial tool. It takes far less time to get information quickly that you?€?d otherwise have to source from stack-overflow, various red-hat articles, Ubuntu articles, searching through software documentation, Microsoft documentation ect. Typically chat gpt can find the answer in a fraction of a second that google can. Obviously it is wrong, a lot. But to have the ability to get quick information on my phone like I can in the web browser I?€?m super excited about and have already been using the mobile app since download constantly. And I?€?m excited for the future of this program becoming more accurate and it seems to be getting more and more precise with every roll out. Gone are the days scouring the internet for obscure pieces of information, chat gpt can find it for you with 2 or 3 prompts. I love this app and I?€?m so happy it?€?s on mobile now. The UI is also very sleek, easy to use. My only complaint with the interface is the history tab at the top right. I actually prefer the conversation tabs on the left in the web app but I understand it would make the app kind of clunky especially on mobile since the screen size is smaller. Anyway, awesome app 5 stars.",4
2,5/19/2023 10:16,"Almost 5 stars, but?€? no search function","This app would almost be perfect if it wasn?€?t for ONE little thing: a ?€?search in?€? function. As anyone can imagine, these AI chats can get quuuuite long, and quite lengthy. And sometimes I wanna go into a chat & look up a specific part or parts of that particular chat by using a search function to look up key words within that chat & track down whatever part I was looking for. For example, in a chat, if I had searched way early into the chat ?€?how do lions hunt??€? And say days later, I wanted to revisit that particular response, I wanna be able to go into the actual chat go to a ?€?search in?€? function and be able to type in key words like ?€?lions?€? or ?€?hunt?€? to be able to automatically find that part in the chat instead of having to scroll through a massive chat to find that part. Similar to what you can do in Microsoft Word docs, or even on web browsers. I think the app already kind of has this, but

In [6]:
chatgpt_train.columns

Index(['date', 'title', 'review', 'rating'], dtype='object')

In [7]:
chatgpt_test = pd.read_csv("chatgpt_test.csv")
chatgpt_test.head()

,date,title,review,rating
0,5/19/2023 6:09,error unsupported country,cant login,2
1,5/19/2023 9:39,Hype junk,More harm than help.,1
2,5/19/2023 4:12,your gpt 4 is fake,Fix it,1
3,5/20/2023 3:01,Please impose IPadOS,We need IPadOS!!!,5
4,5/19/2023 20:49,Amazing,Great,5


### Number of instances in the training dataset with blank reviews

In [8]:
len(chatgpt_train)

1834

### Number of instances in the test dataset with blank reviews

In [9]:
len(chatgpt_test)

458

### Remove blank reviews from training and test dataset

In [10]:
chatgpt_train.dropna(subset = ['review'], inplace=True)
chatgpt_train.head()

,date,title,review,rating
0,5/21/2023 16:42,Much more accessible for blind users than the web version,"Up to this point I?€?ve mostly been using ChatGPT on my windows desktop using Google Chrome. While it?€?s doable, screen reader navigation is pretty difficult on the desktop site and you really have to be an advanced user to find your way through it. I have submitted numerous feedbacks to open AI about this but nothing has changed on that front.\nWell, the good news ?€? the iOS app pretty much addresses all of those problems. The UI seems really clean, uncluttered and designed well to be compatible with voiceover, the screen reader built into iOS. I applaud the inclusivity of this design ?€? I only wish they would give the same attention and care to the accessibility experience of the desktop app.\nI would have given this review five stars but I have just a couple minor quibbles. First, once I submit my prompt, voiceover starts to read aloud ChatGPT?€?s response before that response is finished, so I will hear the first few words of the response followed by voiceover reading aloud the ?€?stop generating?€? button, which isn?€?t super helpful. It would be great if you could better coordinate this alert so that it didn?€?t start reading the message until it had been fully generated. The other thing I?€?d like is a Feedback button easily accessible from within the main screen of the app, to make it as easy as possible to get continuing suggestions and feedback from your users.\nOtherwise, fantastic app so far!",4
1,7/11/2023 12:24,"Much anticipated, wasn?€?t let down.","I?€?ve been a user since it?€?s initial roll out and have been waiting for a mobile application ever since using the web app. For reference I?€?m a software engineering student while working in IT full time. I have to say GPT is an crucial tool. It takes far less time to get information quickly that you?€?d otherwise have to source from stack-overflow, various red-hat articles, Ubuntu articles, searching through software documentation, Microsoft documentation ect. Typically chat gpt can find the answer in a fraction of a second that google can. Obviously it is wrong, a lot. But to have the ability to get quick information on my phone like I can in the web browser I?€?m super excited about and have already been using the mobile app since download constantly. And I?€?m excited for the future of this program becoming more accurate and it seems to be getting more and more precise with every roll out. Gone are the days scouring the internet for obscure pieces of information, chat gpt can find it for you with 2 or 3 prompts. I love this app and I?€?m so happy it?€?s on mobile now. The UI is also very sleek, easy to use. My only complaint with the interface is the history tab at the top right. I actually prefer the conversation tabs on the left in the web app but I understand it would make the app kind of clunky especially on mobile since the screen size is smaller. Anyway, awesome app 5 stars.",4
2,5/19/2023 10:16,"Almost 5 stars, but?€? no search function","This app would almost be perfect if it wasn?€?t for ONE little thing: a ?€?search in?€? function. As anyone can imagine, these AI chats can get quuuuite long, and quite lengthy. And sometimes I wanna go into a chat & look up a specific part or parts of that particular chat by using a search function to look up key words within that chat & track down whatever part I was looking for. For example, in a chat, if I had searched way early into the chat ?€?how do lions hunt??€? And say days later, I wanted to revisit that particular response, I wanna be able to go into the actual chat go to a ?€?search in?€? function and be able to type in key words like ?€?lions?€? or ?€?hunt?€? to be able to automatically find that part in the chat instead of having to scroll through a massive chat to find that part. Similar to what you can do in Microsoft Word docs, or even on web browsers. I think the app already kind of has this, but

In [11]:
chatgpt_test.dropna(subset = ['review'], inplace=True)
chatgpt_test.head()

,date,title,review,rating
0,5/19/2023 6:09,error unsupported country,cant login,2
1,5/19/2023 9:39,Hype junk,More harm than help.,1
2,5/19/2023 4:12,your gpt 4 is fake,Fix it,1
3,5/20/2023 3:01,Please impose IPadOS,We need IPadOS!!!,5
4,5/19/2023 20:49,Amazing,Great,5


### Number of instances in the training dataset with blank reviews removed

In [12]:
len(chatgpt_train)

1829

### Number of instances in the test dataset with blank reviews removed

In [13]:
len(chatgpt_test)

458

## Task 2: POS Tagging

### Copy of the training dataset

In [14]:
chatgpt_train_copy = chatgpt_train.copy()

In [15]:
chatgpt_train_copy

,date,title,review,rating
0,5/21/2023 16:42,Much more accessible for blind users than the web version,"Up to this point I?€?ve mostly been using ChatGPT on my windows desktop using Google Chrome. While it?€?s doable, screen reader navigation is pretty difficult on the desktop site and you really have to be an advanced user to find your way through it. I have submitted numerous feedbacks to open AI about this but nothing has changed on that front.\nWell, the good news ?€? the iOS app pretty much addresses all of those problems. The UI seems really clean, uncluttered and designed well to be compatible with voiceover, the screen reader built into iOS. I applaud the inclusivity of this design ?€? I only wish they would give the same attention and care to the accessibility experience of the desktop app.\nI would have given this review five stars but I have just a couple minor quibbles. First, once I submit my prompt, voiceover starts to read aloud ChatGPT?€?s response before that response is finished, so I will hear the first few words of the response followed by voiceover reading aloud the ?€?stop generating?€? button, which isn?€?t super helpful. It would be great if you could better coordinate this alert so that it didn?€?t start reading the message until it had been fully generated. The other thing I?€?d like is a Feedback button easily accessible from within the main screen of the app, to make it as easy as possible to get continuing suggestions and feedback from your users.\nOtherwise, fantastic app so far!",4
1,7/11/2023 12:24,"Much anticipated, wasn?€?t let down.","I?€?ve been a user since it?€?s initial roll out and have been waiting for a mobile application ever since using the web app. For reference I?€?m a software engineering student while working in IT full time. I have to say GPT is an crucial tool. It takes far less time to get information quickly that you?€?d otherwise have to source from stack-overflow, various red-hat articles, Ubuntu articles, searching through software documentation, Microsoft documentation ect. Typically chat gpt can find the answer in a fraction of a second that google can. Obviously it is wrong, a lot. But to have the ability to get quick information on my phone like I can in the web browser I?€?m super excited about and have already been using the mobile app since download constantly. And I?€?m excited for the future of this program becoming more accurate and it seems to be getting more and more precise with every roll out. Gone are the days scouring the internet for obscure pieces of information, chat gpt can find it for you with 2 or 3 prompts. I love this app and I?€?m so happy it?€?s on mobile now. The UI is also very sleek, easy to use. My only complaint with the interface is the history tab at the top right. I actually prefer the conversation tabs on the left in the web app but I understand it would make the app kind of clunky especially on mobile since the screen size is smaller. Anyway, awesome app 5 stars.",4
2,5/19/2023 10:16,"Almost 5 stars, but?€? no search function","This app would almost be perfect if it wasn?€?t for ONE little thing: a ?€?search in?€? function. As anyone can imagine, these AI chats can get quuuuite long, and quite lengthy. And sometimes I wanna go into a chat & look up a specific part or parts of that particular chat by using a search function to look up key words within that chat & track down whatever part I was looking for. For example, in a chat, if I had searched way early into the chat ?€?how do lions hunt??€? And say days later, I wanted to revisit that particular response, I wanna be able to go into the actual chat go to a ?€?search in?€? function and be able to type in key words like ?€?lions?€? or ?€?hunt?€? to be able to automatically find that part in the chat instead of having to scroll through a massive chat to find that part. Similar to what you can do in Microsoft Word docs, or even on web browsers. I think the app already kind of has this, but

In [16]:
chatgpt_train_copy.loc[:, ('tokenized_review')] = chatgpt_train_copy.loc[:, ('review')].str.lower().apply(nltk.word_tokenize)

In [17]:
chatgpt_train_copy[['tokenized_review']]

,tokenized_review
0,"[up, to, this, point, i, ?, €, ?, ve, mostly, been, using, chatgpt, on, my, windows, desktop, using, google, chrome, ., while, it, ?, €, ?, s, doable, ,, screen, reader, navigation, is, pretty, difficult, on, the, desktop, site, and, you, really, have, to, be, an, advanced, user, to, find, your, way, through, it, ., i, have, submitted, numerous, feedbacks, to, open, ai, about, this, but, nothing, has, changed, on, that, front, ., well, ,, the, good, news, ?, €, ?, the, ios, app, pretty, much, addresses, all, of, those, problems, ., the, ui, seems, really, clean, ,, uncluttered, and, ...]"
1,"[i, ?, €, ?, ve, been, a, user, since, it, ?, €, ?, s, initial, roll, out, and, have, been, waiting, for, a, mobile, application, ever, since, using, the, web, app, ., for, reference, i, ?, €, ?, m, a, software, engineering, student, while, working, in, it, full, time, ., i, have, to, say, gpt, is, an, crucial, tool, ., it, takes, far, less, time, to, get, information, quickly, that, you, ?, €, ?, d, otherwise, have, to, source, from, stack-overflow, ,, various, red-hat, articles, ,, ubuntu, articles, ,, searching, through, software, documentation, ,, microsoft, documentation, ect, ., typically, chat, ...]"
2,"[this, app, would, almost, be, perfect, if, it, wasn, ?, €, ?, t, for, one, little, thing, :, a, ?, €, ?, search, in, ?, €, ?, function, ., as, anyone, can, imagine, ,, these, ai, chats, can, get, quuuuite, long, ,, and, quite, lengthy, ., and, sometimes, i, wan, na, go, into, a, chat, &, look, up, a, specific, part, or, parts, of, that, particular, chat, by, using, a, search, function, to, look, up, key, words, within, that, chat, &, track, down, whatever, part, i, was, looking, for, ., for, example, ,, in, a, chat, ,, if, i, had, ...]"
3,"[i, recently, downloaded, the, app, and, overall, ,, it, 's, a, great, platform, with, excellent, potential, ., however, ,, i, did, encounter, a, couple, of, issues, with, logging, in, that, i, feel, need, to, be, addressed, ., firstly, ,, the, login, process, was, somewhat, cumbersome, ., it, took, me, a, few, attempts, to, successfully, log, in, ,, as, the, app, did, n't, always, recognize, my, credentials, right, away, ., this, could, be, improved, by, streamlining, the, login, flow, and, ensuring, a, smoother, user, experience, ., secondly, ,, the, app, occasionally, experienced, login, glitches, ,, where, it, would, unexpectedly, log, me, ...]"
4,"[i, appreciate, the, devs, implementing, siri, support, ?, €, ?, it, is, already, enhancing, the, usefulness, of, the, app, ,, despite, being, a, little, clunky, ., i, ?, €, ?, d, prefer, if, it, were, possible, to, make, a, query, in, one, fell, swoop, ,, however, ., currently, ,, it, seems, like, i, have, to, say, ?, €, ?, ask, chatgpt, ?, €, ?, then, wait, to, be, asked, what, my, query, is, before, saying, the, actual, query, ., i, know, that, it, ?, €, ?, s, possible, in, other, contexts, to, submit, a, request, to, a, third-party, app, through, ...]"
...,...
1829,"[please, make, a, ipad, version, of, this]"
1830,"[best, app]"
1831,"[why, it, ?, €, ?, s, not, available, in, ethiopia]"
1832,"[it, agrees, with, letting, children, be, forced, into, pride, or, homosexuality]"


In [18]:
chatgpt_train_copy.loc[:, ('pos_tags')] = chatgpt_train_copy.loc[:, ('tokenized_review')].apply(nltk.pos_tag)

In [19]:
chatgpt_train_copy[['pos_tags']]

,pos_tags
0,"[(up, RB), (to, TO), (this, DT), (point, NN), (i, NN), (?, .), (€, NN), (?, .), (ve, NN), (mostly, RB), (been, VBN), (using, VBG), (chatgpt, NN), (on, IN), (my, PRP$), (windows, NNS), (desktop, VBP), (using, VBG), (google, NN), (chrome, NN), (., .), (while, IN), (it, PRP), (?, .), (€, VB), (?, .), (s, NN), (doable, JJ), (,, ,), (screen, JJ), (reader, NN), (navigation, NN), (is, VBZ), (pretty, RB), (difficult, JJ), (on, IN), (the, DT), (desktop, NN), (site, NN), (and, CC), (you, PRP), (really, RB), (have, VB), (to, TO), (be, VB), (an, DT), (advanced, JJ), (user, NN), (to, TO), (find, VB), (your, PRP$), (way, NN), (through, IN), (it, PRP), (., .), (i, NNS), (have, VBP), (submitted, VBN), (numerous, JJ), (feedbacks, NNS), (to, TO), (open, VB), (ai, NN), (about, IN), (this, DT), (but, CC), (nothing, NN), (has, VBZ), (changed, VBN), (on, IN), (that, DT), (front, NN), (., .), (well, RB), (,, ,), (the, DT), (good, JJ), (news, NN), (?, .), (€, NN), (?, .), (the, DT), (ios, NNS), (app, VBP), (pretty, RB), (much, JJ), (addresses, VBZ), (all, DT), (of, IN), (those, DT), (problems, NNS), (., .), (the, DT), (ui, JJ), (seems, VBZ), (really, RB), (clean, JJ), (,, ,), (uncluttered, JJ), (and, CC), ...]"
1,"[(i, NN), (?, .), (€, NN), (?, .), (ve, RB), (been, VBN), (a, DT), (user, NN), (since, IN), (it, PRP), (?, .), (€, VB), (?, .), (s, JJ), (initial, JJ), (roll, NN), (out, RB), (and, CC), (have, VBP), (been, VBN), (waiting, VBG), (for, IN), (a, DT), (mobile, JJ), (application, NN), (ever, RB), (since, IN), (using, VBG), (the, DT), (web, NN), (app, NN), (., .), (for, IN), (reference, NN), (i, NN), (?, .), (€, NN), (?, .), (m, VB), (a, DT), (software, NN), (engineering, NN), (student, NN), (while, IN), (working, VBG), (in, IN), (it, PRP), (full, JJ), (time, NN), (., .), (i, NNS), (have, VBP), (to, TO), (say, VB), (gpt, NN), (is, VBZ), (an, DT), (crucial, JJ), (tool, NN), (., .), (it, PRP), (takes, VBZ), (far, RB), (less, JJR), (time, NN), (to, TO), (get, VB), (information, NN), (quickly, RB), (that, IN), (you, PRP), (?, .), (€, VB), (?, .), (d, JJ), (otherwise, RB), (have, VBP), (to, TO), (source, NN), (from, IN), (stack-overflow, JJ), (,, ,), (various, JJ), (red-hat, JJ), (articles, NNS), (,, ,), (ubuntu, JJ), (articles, NNS), (,, ,), (searching, VBG), (through, IN), (software, NN), (documentation, NN), (,, ,), (microsoft, JJ), (documentation, NN), (ect, NN), (., .), (typically, RB), (chat, WP), ...]"
2,"[(this, DT), (app, NN), (would, MD), (almost, RB), (be, VB), (perfect, JJ), (if, IN), (it, PRP), (wasn, VBZ), (?, .), (€, VB), (?, .), (t, NN), (for, IN), (one, CD), (little, JJ), (thing, NN), (:, :), (a, DT), (?, .), (€, NN), (?, .), (search, NN), (in, IN), (?, .), (€, NN), (?, .), (function, NN), (., .), (as, IN), (anyone, NN), (can, MD), (imagine, VB), (,, ,), (these, DT), (ai, VBP), (chats, NNS), (can, MD), (get, VB), (quuuuite, RB), (long, RB), (,, ,), (and, CC), (quite, RB), (lengthy, JJ), (., .), (and, CC), (sometimes, RB), (i, JJ), (wan, VBP), (na, TO), (go, VB), (into, IN), (a, DT), (chat, NN), (&, CC), (look, VB), (up, RP), (a, DT), (specific, JJ), (part, NN), (or, CC), (parts, NNS), (of, IN), (that, DT), (particular, JJ), (chat, NN), (by, IN), (using, VBG), (a, DT), (search, NN), (function, NN), (to, TO), (look, VB), (up, RP), (key, JJ), (words, NNS), (within, IN), (that, DT), (chat, NN), (&, CC), (track, VB), (down, RP), (whatever, WDT), (part, NN), (i, NN), (was, VBD), (looking, VBG), (for, IN), (., .), (for, IN), (example, NN), (,, ,), (in, IN), (a, DT), (chat, NN), (,, ,), (if, IN), (i, VBN), (had, VBD), ...]"
3,"[(i, NN), (recently, RB), (downloaded, VBD), (the, DT), (app, NN), (and, CC), (overall, JJ), (,, ,), (it, PRP), ('s, VBZ), (a, DT), (great, JJ), (platform, NN), (with, IN), (excellent, JJ), (potential, NN), (., .), (however, RB), (,, ,), (i, NN), (did, VBD), (encounter, VB), (a, DT), (couple, NN), (of, IN), (issues, NNS), (with, IN), (logging, VBG), (in, IN), (that, DT), (i, NN), (feel, VBP), (need, NN), (t

### 3 examples of review and POS Tagging

In [20]:
chatgpt_train_copy.loc[35:35, ('review', 'pos_tags')]

,review,pos_tags
35,"With the exceptional performance seen on the browser of desktop, I was unsure as to UI on mobile. I can say that it?€?s even more intuitive while trying to navigate the now native app. \n\nThe haptic feedback is especially unique and engaging with responses while being generated. \n\nI look forward to future improvements and abilities that ChatGPT will be able to integrate into beyond the sandbox. I do hope that once certain barriers are met conversations can move past the guard rails. \n\nImprovements; Sharing threads you create with colleagues outside of screenshots would be ideal for broadening collaboration. Hypothetical War-games, strategy and the like while having a conversation in a group thread on Game Theory would be ideal. \n\nWith the ability of inputting other app?€?s Terms of Services (Tik-Tok May 19, 2023) for example; prior to agreeing to them, the ability to compare and contrast with the previous terms would be helpful to cut out the mindless jargon. ChatGPT already has this ability, now that it?€?s native to iOS. \n\nHaving it the past several days, I?€?ve already seen an update, with some refreshed improvements. If this?€? OpenAi?€?s platform aperitif, I very much look forward to the coming entr??e.","[(with, IN), (the, DT), (exceptional, JJ), (performance, NN), (seen, VBN), (on, IN), (the, DT), (browser, NN), (of, IN), (desktop, NN), (,, ,), (i, NN), (was, VBD), (unsure, JJ), (as, IN), (to, TO), (ui, VB), (on, IN), (mobile, NN), (., .), (i, NN), (can, MD), (say, VB), (that, IN), (it, PRP), (?, .), (€, VB), (?, .), (s, VB), (even, RB), (more, RBR), (intuitive, JJ), (while, IN), (trying, VBG), (to, TO), (navigate, VB), (the, DT), (now, RB), (native, JJ), (app, NN), (., .), (the, DT), (haptic, JJ), (feedback, NN), (is, VBZ), (especially, RB), (unique, JJ), (and, CC), (engaging, VBG), (with, IN), (responses, NNS), (while, IN), (being, VBG), (generated, VBN), (., .), (i, JJ), (look, VBP), (forward, RB), (to, TO), (future, JJ), (improvements, NNS), (and, CC), (abilities, NNS), (that, IN), (chatgpt, NN), (will, MD), (be, VB), (able, JJ), (to, TO), (integrate, VB), (into, IN), (beyond, IN), (the, DT), (sandbox, NN), (., .), (i, VB), (do, VBP), (hope, VB), (that, DT), (once, RB), (certain, JJ), (barriers, NNS), (are, VBP), (met, VBN), (conversations, NNS), (can, MD), (move, VB), (past, IN), (the, DT), (guard, NN), (rails, VBZ), (., .), (improvements, NNS), (;, :), (sharing, VBG), (threads, NNS), (you, PRP), (create, VBP), (with, IN), (colleagues, NNS), ...]"


In [21]:
chatgpt_train_copy.loc[100:100, ('review', 'pos_tags')]

,review,pos_tags
100,"Simple and not complicated at all user interface. Easy to navigate around and overall a good user experience. However, there is no option to generate a different response to the same prompt and to edit a prompt I wrote like how you would in the web version. In my opinion, I would still use the web version due to those features. The mobile version would be better for quicker access and response.","[(simple, NN), (and, CC), (not, RB), (complicated, VBN), (at, IN), (all, DT), (user, JJ), (interface, NN), (., .), (easy, JJ), (to, TO), (navigate, VB), (around, IN), (and, CC), (overall, VB), (a, DT), (good, JJ), (user, NN), (experience, NN), (., .), (however, RB), (,, ,), (there, EX), (is, VBZ), (no, DT), (option, NN), (to, TO), (generate, VB), (a, DT), (different, JJ), (response, NN), (to, TO), (the, DT), (same, JJ), (prompt, NN), (and, CC), (to, TO), (edit, VB), (a, DT), (prompt, NN), (i, JJ), (wrote, VBD), (like, IN), (how, WRB), (you, PRP), (would, MD), (in, IN), (the, DT), (web, JJ), (version, NN), (., .), (in, IN), (my, PRP$), (opinion, NN), (,, ,), (i, NN), (would, MD), (still, RB), (use, VB), (the, DT), (web, NN), (version, NN), (due, JJ), (to, TO), (those, DT), (features, NNS), (., .), (the, DT), (mobile, JJ), (version, NN), (would, MD), (be, VB), (better, JJR), (for, IN), (quicker, JJR), (access, NN), (and, CC), (response, NN), (., .)]"


In [22]:
chatgpt_train_copy.loc[1500:1500, ('review', 'pos_tags')]

,review,pos_tags
1500,The ?€?very simplistic?€? ui hits a sweet spot for me ????,"[(the, DT), (?, .), (€, NN), (?, .), (very, RB), (simplistic, JJ), (?, .), (€, NN), (?, .), (ui, JJ), (hits, VBZ), (a, DT), (sweet, JJ), (spot, NN), (for, IN), (me, PRP), (?, .), (?, .), (?, .), (?, .)]"


## Task 3: Extract unigram features

### Fit training dataset

In [23]:
# range is (1,1) for extracting unigrams
vectorizer = CountVectorizer(ngram_range = (1,1))

# tokenize and build vocabulary
vectorizer.fit(chatgpt_train['review'])
print(vectorizer.vocabulary_, '\n')

# create feature vector representation
vector = vectorizer.transform(chatgpt_train['review'])

print(vector.shape,"\n") 

# complete vectors
print(vector.toarray())

{'up': 5224, 'to': 4985, 'this': 4932, 'point': 3613, 've': 5282, 'mostly': 3134, 'been': 493, 'using': 5253, 'chatgpt': 776, 'on': 3331, 'my': 3165, 'windows': 5454, 'desktop': 1302, 'google': 2139, 'chrome': 806, 'while': 5421, 'it': 2666, 'doable': 1426, 'screen': 4267, 'reader': 3883, 'navigation': 3184, 'is': 2660, 'pretty': 3692, 'difficult': 1352, 'the': 4906, 'site': 4443, 'and': 256, 'you': 5540, 'really': 3891, 'have': 2248, 'be': 474, 'an': 250, 'advanced': 151, 'user': 5250, 'find': 1907, 'your': 5542, 'way': 5382, 'through': 4954, 'submitted': 4698, 'numerous': 3277, 'feedbacks': 1877, 'open': 3341, 'ai': 197, 'about': 60, 'but': 667, 'nothing': 3255, 'has': 2241, 'changed': 754, 'that': 4903, 'front': 2032, 'well': 5403, 'good': 2135, 'news': 3221, 'ios': 2640, 'app': 291, 'much': 3147, 'addresses': 137, 'all': 213, 'of': 3303, 'those': 4937, 'problems': 3720, 'ui': 5130, 'seems': 4306, 'clean': 821, 'uncluttered': 5148, 'designed': 1295, 'compatible': 907, 'with': 5464, 

### Fit the test dataset

In [24]:
# range is (1,1) for extracting unigrams
vectorizer = CountVectorizer(ngram_range = (1,1))

# tokenize and build vocabulary
vectorizer.fit(chatgpt_test['review'])
print(vectorizer.vocabulary_, '\n')

# create feature vector representation
vector = vectorizer.transform(chatgpt_test['review'])

print(vector.shape,"\n") 

# complete vectors
print(vector.toarray())

{'cant': 271, 'login': 1012, 'more': 1101, 'harm': 776, 'than': 1714, 'help': 790, 'fix': 662, 'it': 923, 'we': 1884, 'need': 1123, 'ipados': 914, 'great': 741, 'this': 1734, 'is': 918, 'really': 1382, 'good': 731, 'app': 140, 'first': 660, 'official': 1170, 'gpt': 735, 'or': 1190, 'old': 1174, 'version': 1857, 'of': 1166, 'chatgpt': 306, 'unusable': 1812, 'on': 1175, 'ipad': 913, 'pro': 1314, 'currently': 430, 'what': 1889, 'the': 1718, 'title': 1757, 'says': 1492, 'subscribe': 1653, 'plus': 1272, 'for': 672, 'gpt4': 737, 'but': 255, 'just': 935, 'get': 714, 'gpt3': 736, 'with': 1911, 'fake': 634, 'label': 950, 'release': 1412, 'real': 1380, 'now': 1156, 'looks': 1022, 'nice': 1134, 'to': 1758, 'comment': 342, 'thanks': 1716, 'openai': 1184, 'wants': 1878, 'not': 1151, 'only': 1180, 'an': 118, 'sso': 1617, 'sign': 1565, 'in': 860, 'account': 58, 'also': 104, 'name': 1116, 'birth': 223, 'telephone': 1700, 'number': 1159, 'and': 119, 'don': 504, 'know': 945, 'else': 544, 'because': 202,

In [25]:
# use review for model
train_text = chatgpt_train["review"]
test_text = chatgpt_test["review"]

# set the n-gram range
vectorizer = CountVectorizer(ngram_range = (1,1))

# create training data representation
train_data_repr = vectorizer.fit_transform(train_text.values.astype('U'))
print("NUMBER OF FEATURES IN TRAINING DATASET WITH UNIGRAMS")
print(train_data_repr.shape,"\n") 

# create test data representation
test_data_repr = vectorizer.transform(test_text.values.astype('U'))
print("NUMBER OF FEATURES IN TEST DATASET WITH UNIGRAMS")
print(test_data_repr.shape,"\n") 

NUMBER OF FEATURES IN TRAINING DATASET WITH UNIGRAMS
(1829, 5551) 

NUMBER OF FEATURES IN TEST DATASET WITH UNIGRAMS
(458, 5551) 



## Task 4: Training and evaluate Naive Bayes classifier with unigram features

In [26]:
# define true labels from train set
X_train = train_data_repr
y_train = chatgpt_train["rating"]
X_test = test_data_repr
y_test = chatgpt_test["rating"]

In [27]:
# Naive Bayes model
model_unigram = MultinomialNB()
model_unigram.fit(X_train, y_train)

MultinomialNB()

In [28]:
# predict the labels for the test data
predictions = model_unigram.predict(X_test)
predictions

array([1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 3, 1, 1, 5, 5, 5, 1, 1, 5, 5, 5,
       1, 5, 5, 1, 5, 5, 1, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 1, 3, 5, 5,
       5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 1, 5, 3, 4, 5, 5, 1, 5, 5, 1, 1, 5,
       5, 5, 5, 1, 5, 4, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 5, 5,
       5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5,
       1, 1, 1, 5, 1, 1, 5, 5, 5, 5, 5, 2, 2, 5, 3, 1, 5, 5, 5, 1, 5, 5,
       5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 2, 1, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,
       5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 1, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 1, 3, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 5, 5, 5, 5,
       1, 4, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 1, 5,

### Saving the model

In [29]:
import pickle

filename_1 = 'NaiveBayesModel_unigram.sav'
pickle.dump(model_unigram, open(filename_1, 'wb'))

### Load the model

In [30]:
loaded_model_unigram = pickle.load(open(filename_1, 'rb'))
result = loaded_model_unigram.score(X_test, y_test)
print(result)

0.6331877729257642


### Evaluating the unigram model using metrics

In [31]:
print ("Overall Accuracy score: ", accuracy_score(y_test, predictions))
print ("Overall Recall score: ", recall_score(y_test, predictions, average='macro'))
print ("Overall Precision score: ", precision_score(y_test, predictions, average='macro'))
print ("Overall F1 score: ", f1_score(y_test, predictions, average='macro'), "\n")
print ("Individual label performance: ")
print (classification_report(y_test, predictions))
print ("Confusion Matrix: ")
print (confusion_matrix(y_test, predictions))

Overall Accuracy score:  0.6331877729257642
Overall Recall score:  0.34043281122804264
Overall Precision score:  0.4365165675446049
Overall F1 score:  0.35240689572225525 

Individual label performance: 
              precision    recall  f1-score   support

           1       0.57      0.49      0.52       115
           2       0.40      0.08      0.13        26
           3       0.13      0.08      0.10        25
           4       0.39      0.16      0.23        44
           5       0.69      0.90      0.78       248

    accuracy                           0.63       458
   macro avg       0.44      0.34      0.35       458
weighted avg       0.59      0.63      0.59       458

Confusion Matrix: 
[[ 56   0   6   4  49]
 [ 13   2   1   1   9]
 [  8   0   2   2  13]
 [  6   0   4   7  27]
 [ 16   3   2   4 223]]


## Task 5: Add bigram features

In [32]:
# range is (1,2) for extracting unigrams and bigrams
vectorizer = CountVectorizer(ngram_range = (1,2))

# tokenize and build vocabulary
vectorizer.fit(chatgpt_train['review'])
print(vectorizer.vocabulary_, '\n')

# create feature vector representation
vector = vectorizer.transform(chatgpt_train['review'])

print(vector.shape,"\n") 

# complete vectors
print(vector.toarray())

{'up': 37347, 'to': 35638, 'this': 35036, 'point': 26826, 've': 38075, 'mostly': 22562, 'been': 5228, 'using': 37876, 'chatgpt': 7119, 'on': 24675, 'my': 22722, 'windows': 39767, 'desktop': 9651, 'google': 14656, 'chrome': 7449, 'while': 39475, 'it': 18831, 'doable': 10297, 'screen': 29943, 'reader': 28240, 'navigation': 23095, 'is': 18313, 'pretty': 27163, 'difficult': 10007, 'the': 33738, 'site': 31021, 'and': 2014, 'you': 40659, 'really': 28300, 'have': 15450, 'be': 4887, 'an': 1842, 'advanced': 888, 'user': 37745, 'find': 12881, 'your': 40851, 'way': 38846, 'through': 35416, 'submitted': 32364, 'numerous': 23978, 'feedbacks': 12668, 'open': 25051, 'ai': 1105, 'about': 245, 'but': 6177, 'nothing': 23775, 'has': 15322, 'changed': 6939, 'that': 33429, 'front': 13871, 'well': 39085, 'good': 14581, 'news': 23360, 'ios': 18072, 'app': 3207, 'much': 22599, 'addresses': 835, 'all': 1313, 'of': 24053, 'those': 35291, 'problems': 27335, 'ui': 36999, 'seems': 30277, 'clean': 7481, 'uncluttere

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### Number of features in the dataset using unigrams and bigrams

In [33]:
# use review for model building

train_text = chatgpt_train["review"]
test_text = chatgpt_test["review"]

# set the n-gram range
vectorizer = CountVectorizer(ngram_range = (1,2))

# create training data representation
train_data_repr = vectorizer.fit_transform(train_text.values.astype('U'))
print("NUMBER OF FEATURES IN TRAINING DATASET WITH UNIGRAMS, BIGRAMS")
print(train_data_repr.shape,"\n") 

# create test data representation
test_data_repr = vectorizer.transform(test_text.values.astype('U'))
print("NUMBER OF FEATURES IN TEST DATASET WITH UNIGRAMS, BIGRAMS")
print(test_data_repr.shape,"\n") 

NUMBER OF FEATURES IN TRAINING DATASET WITH UNIGRAMS, BIGRAMS
(1829, 41006) 

NUMBER OF FEATURES IN TEST DATASET WITH UNIGRAMS, BIGRAMS
(458, 41006) 



### Train Naive Bayes with unigram and bigram features

In [34]:
from sklearn.naive_bayes import MultinomialNB

# define true labels from train set
X_train = train_data_repr
y_train = chatgpt_train["rating"]
X_test = test_data_repr
y_test = chatgpt_test["rating"]

In [35]:
# Naive Bayes model
model_unigram_bigram = MultinomialNB()
model_unigram_bigram.fit(X_train, y_train)

MultinomialNB()

In [36]:
# predict the labels for the test data
predictions = model_unigram_bigram.predict(X_test)
predictions

array([1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 3, 1, 1, 4, 5, 5, 5, 1, 5, 5, 5,
       1, 5, 5, 1, 5, 5, 1, 5, 5, 5, 5, 5, 1, 5, 1, 5, 5, 5, 1, 5, 5, 5,
       5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 4, 5, 5, 1, 5, 5, 4, 1, 5,
       5, 5, 5, 1, 5, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5,
       1, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5,
       5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 4, 1, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,
       5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,

### Saving the unigram + bigram model

In [37]:
import pickle

filename_2 = 'NaiveBayesModel_unigram_bigram.sav'
pickle.dump(model_unigram_bigram, open(filename_2, 'wb'))

### Loading the unigram + bigram model

In [38]:
loaded_model_unigram_bigram = pickle.load(open(filename_2, 'rb'))
result = loaded_model_unigram_bigram.score(X_test, y_test)
print(result)

0.6244541484716157


### Evaluating unigram + bigram model using metrics

In [39]:
print ("Accuracy score: ", accuracy_score(y_test, predictions))
print ("Overall Recall score: ", recall_score(y_test, predictions, average='macro'))
print ("Overall Precision score: ", precision_score(y_test, predictions, average='macro'))
print ("Overall F1 score: ", f1_score(y_test, predictions, average='macro'), "\n")
print ("Individual label performance: ")
print (classification_report(y_test, predictions), "\n")
print ("Confusion Matrix: ")
print (confusion_matrix(y_test, predictions))

Accuracy score:  0.6244541484716157
Overall Recall score:  0.28008542649496365
Overall Precision score:  0.27141238135713275
Overall F1 score:  0.26332604636538254 

Individual label performance: 
              precision    recall  f1-score   support

           1       0.63      0.43      0.51       115
           2       0.00      0.00      0.00        26
           3       0.00      0.00      0.00        25
           4       0.08      0.02      0.04        44
           5       0.65      0.95      0.77       248

    accuracy                           0.62       458
   macro avg       0.27      0.28      0.26       458
weighted avg       0.52      0.62      0.55       458
 

Confusion Matrix: 
[[ 49   0   3   3  60]
 [ 11   0   1   3  11]
 [  7   0   0   0  18]
 [  5   0   1   1  37]
 [  6   0   0   6 236]]


/opt/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/

## Task 6: Add trigram features

In [40]:
# range is (1,3) for extracting unigrams, bigrams, and trigrams
vectorizer = CountVectorizer(ngram_range = (1,3))

# tokenize and build vocabulary
vectorizer.fit(chatgpt_train['review'])
print(vectorizer.vocabulary_, '\n')

# create feature vector representation
vector = vectorizer.transform(chatgpt_train['review'])

print(vector.shape,"\n") 

# complete vectors
print(vector.toarray())

{'up': 88181, 'to': 83615, 'this': 81998, 'point': 63324, 've': 90083, 'mostly': 53192, 'been': 13161, 'using': 89621, 'chatgpt': 17504, 'on': 58432, 'my': 53540, 'windows': 94178, 'desktop': 22813, 'google': 33804, 'chrome': 18396, 'while': 93506, 'it': 44026, 'doable': 24127, 'screen': 69557, 'reader': 66160, 'navigation': 54558, 'is': 42397, 'pretty': 63984, 'difficult': 23554, 'the': 77753, 'site': 71859, 'and': 4696, 'you': 96345, 'really': 66300, 'have': 35652, 'be': 12331, 'an': 4140, 'advanced': 1970, 'user': 89238, 'find': 29554, 'your': 97009, 'way': 91835, 'through': 83130, 'submitted': 74618, 'numerous': 56594, 'feedbacks': 29094, 'open': 59443, 'ai': 2388, 'about': 558, 'but': 15060, 'nothing': 56114, 'has': 35314, 'changed': 17059, 'that': 76814, 'front': 32033, 'well': 92506, 'good': 33615, 'news': 55136, 'ios': 41752, 'app': 7980, 'much': 53253, 'addresses': 1872, 'all': 2939, 'of': 56719, 'those': 82872, 'problems': 64337, 'ui': 87498, 'seems': 70303, 'clean': 18445, '

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### Fit Naive Bayes

In [41]:
# use review for model building

train_text = chatgpt_train["review"]
test_text = chatgpt_test["review"]

# set the n-gram range
vectorizer = CountVectorizer(ngram_range = (1,3))

# create training data representation
train_data_repr = vectorizer.fit_transform(train_text.values.astype('U'))
print("NUMBER OF FEATURES IN TRAINING DATASET WITH UNIGRAMS, BIGRAMS, TRIGRAMS")
print(train_data_repr.shape,"\n") 

# create test data representation
test_data_repr = vectorizer.transform(test_text.values.astype('U'))
print("NUMBER OF FEATURES IN TRAINING DATASET WITH UNIGRAMS, BIGRAMS, TRIGRAMS")
print(test_data_repr.shape,"\n") 

NUMBER OF FEATURES IN TRAINING DATASET WITH UNIGRAMS, BIGRAMS, TRIGRAMS
(1829, 97367) 

NUMBER OF FEATURES IN TRAINING DATASET WITH UNIGRAMS, BIGRAMS, TRIGRAMS
(458, 97367) 



In [42]:
from sklearn.naive_bayes import MultinomialNB

# define true labels from train set
X_train = train_data_repr
y_train = chatgpt_train["rating"]
X_test = test_data_repr
y_test = chatgpt_test["rating"]

In [43]:
# Naive Bayes model
model_unigram_bigram_trigram = MultinomialNB()
model_unigram_bigram_trigram.fit(X_train, y_train)

MultinomialNB()

In [44]:
# predict the labels for the test data
predictions = model_unigram_bigram_trigram.predict(X_test)
predictions

array([1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 3, 1, 1, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 1, 5, 5, 5, 1, 5, 5, 5,
       5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 4, 5, 5, 1, 5, 5, 4, 1, 5,
       5, 5, 5, 1, 5, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5,
       1, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 4, 1, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,
       5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,

### Saving the unigram + bigram + trigram model

In [45]:
import pickle

filename_3 = 'NaiveBayesModel_unigram_bigram_trigram.sav'
pickle.dump(model_unigram_bigram_trigram, open(filename_3, 'wb'))

### Loading the unigram + bigram + trigram model

In [46]:
loaded_model_unigram_bigram_trigram = pickle.load(open(filename_3, 'rb'))
result = loaded_model_unigram_bigram_trigram.score(X_test, y_test)
print(result)

0.6200873362445415


### Evaluating unigram + bigram + trigram model using metrics

In [47]:
print ("Accuracy score: ", accuracy_score(y_test, predictions))
print ("Overall Recall score: ", recall_score(y_test, predictions, average='macro'))
print ("Overall Precision score: ", precision_score(y_test, predictions, average='macro'))
print ("Overall F1 score: ", f1_score(y_test, predictions, average='macro'), "\n")
print ("Individual label performance: ")
print (classification_report(y_test, predictions), "\n")
print ("Confusion Matrix: ")
print (confusion_matrix(y_test, predictions))

Accuracy score:  0.6200873362445415
Overall Recall score:  0.270070126227209
Overall Precision score:  0.2570713391739675
Overall F1 score:  0.250021017234132 

Individual label performance: 
              precision    recall  f1-score   support

           1       0.65      0.38      0.48       115
           2       0.00      0.00      0.00        26
           3       0.00      0.00      0.00        25
           4       0.00      0.00      0.00        44
           5       0.64      0.97      0.77       248

    accuracy                           0.62       458
   macro avg       0.26      0.27      0.25       458
weighted avg       0.51      0.62      0.54       458
 

Confusion Matrix: 
[[ 44   0   3   3  65]
 [ 10   0   1   2  13]
 [  6   0   0   0  19]
 [  4   0   1   0  39]
 [  4   0   0   4 240]]


/opt/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/

## Task 7: Add TF-IDF Features

### The Naive Bayes with unigram features has the highest overall F1 score = 0.3524

### Feature Extraction using TfidfVectorizer for training and testing data on Review¶

In [48]:
# Using TF-IDF
text = chatgpt_train["review"]

tf = TfidfVectorizer()
tf.fit(text)

print("VOCABULARY", "\n")
print(tf.vocabulary_,"\n")

# encode document
data = tf.transform(text)

# summarize encoded vector
print("FEATURE SET SHAPE")
print(data.shape,"\n") 

# feature set
print("FEATURE SET ARRAY")
print(data.toarray())

VOCABULARY 

{'up': 5224, 'to': 4985, 'this': 4932, 'point': 3613, 've': 5282, 'mostly': 3134, 'been': 493, 'using': 5253, 'chatgpt': 776, 'on': 3331, 'my': 3165, 'windows': 5454, 'desktop': 1302, 'google': 2139, 'chrome': 806, 'while': 5421, 'it': 2666, 'doable': 1426, 'screen': 4267, 'reader': 3883, 'navigation': 3184, 'is': 2660, 'pretty': 3692, 'difficult': 1352, 'the': 4906, 'site': 4443, 'and': 256, 'you': 5540, 'really': 3891, 'have': 2248, 'be': 474, 'an': 250, 'advanced': 151, 'user': 5250, 'find': 1907, 'your': 5542, 'way': 5382, 'through': 4954, 'submitted': 4698, 'numerous': 3277, 'feedbacks': 1877, 'open': 3341, 'ai': 197, 'about': 60, 'but': 667, 'nothing': 3255, 'has': 2241, 'changed': 754, 'that': 4903, 'front': 2032, 'well': 5403, 'good': 2135, 'news': 3221, 'ios': 2640, 'app': 291, 'much': 3147, 'addresses': 137, 'all': 213, 'of': 3303, 'those': 4937, 'problems': 3720, 'ui': 5130, 'seems': 4306, 'clean': 821, 'uncluttered': 5148, 'designed': 1295, 'compatible': 907, '

### Feature Extraction using TfidfVectorizer for training and testing data on Review

In [49]:
train_data_tfidf = tf.fit_transform(chatgpt_train['review'])
print("TRAINING DATA FEATURE SET SHAPE")
print(train_data_tfidf.shape,"\n") 

test_data_tfidf = tf.transform(chatgpt_test['review'])
print("TEST DATA FEATURE SET SHAPE")
print(test_data_tfidf.shape,"\n") 

idf = tf.idf_

# print out feature names and the IDF values
print("FEATURE SET AND IDF VALUES")
print(dict(zip(tf.get_feature_names(), idf)))

TRAINING DATA FEATURE SET SHAPE
(1829, 5551) 

TEST DATA FEATURE SET SHAPE
(458, 5551) 

FEATURE SET AND IDF VALUES
{'10': 5.567632266669026, '100': 6.902633333401366, '101': 7.818924065275521, '11': 7.125776884715576, '12': 5.804021044733257, '128': 7.818924065275521, '12pro': 7.413458957167357, '13': 6.0271645960474665, '13pro': 7.818924065275521, '14': 5.216234379831137, '15': 6.720311776607412, '156': 7.818924065275521, '16': 5.94712188837393, '17': 7.818924065275521, '18': 7.413458957167357, '19': 7.125776884715576, '1940': 7.818924065275521, '1949': 7.818924065275521, '20': 5.376577029906317, '2007': 7.818924065275521, '2020': 7.818924065275521, '2021': 4.798499179131159, '2022': 6.902633333401366, '2023': 6.432629704155631, '20mins': 7.413458957167357, '21': 7.818924065275521, '23': 7.818924065275521, '24': 7.125776884715576, '25': 6.902633333401366, '27': 7.818924065275521, '2nd': 7.413458957167357, '30': 6.720311776607412, '31': 7.818924065275521, '32k': 7.818924065275521, '35

### Naive Bayes Model using TF-IDF features

In [50]:
# define true labels from train set

X_train = train_data_tfidf
y_train = chatgpt_train["rating"]
X_test = test_data_tfidf
y_test = chatgpt_test["rating"]

In [51]:
model_tfidf = MultinomialNB()
model_tfidf.fit(X_train, y_train)

MultinomialNB()

In [52]:
predictions = model_tfidf.predict(X_test)
predictions

array([1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,

In [53]:
print ("Accuracy score: ", accuracy_score(y_test, predictions))
print ("Overall Recall score: ", recall_score(y_test, predictions, average='macro'))
print ("Overall Precision score: ", precision_score(y_test, predictions, average='macro'))
print ("Overall F1 score: ", f1_score(y_test, predictions, average='macro'), "\n")
print ("Individual label performance: ")
print (classification_report(y_test, predictions), "\n")
print ("Confusion Matrix: ")
print (confusion_matrix(y_test, predictions))

Accuracy score:  0.5829694323144105
Overall Recall score:  0.23397615708274894
Overall Precision score:  0.24875389408099688
Overall F1 score:  0.20132625994694964 

Individual label performance: 
              precision    recall  f1-score   support

           1       0.67      0.17      0.28       115
           2       0.00      0.00      0.00        26
           3       0.00      0.00      0.00        25
           4       0.00      0.00      0.00        44
           5       0.58      1.00      0.73       248

    accuracy                           0.58       458
   macro avg       0.25      0.23      0.20       458
weighted avg       0.48      0.58      0.46       458
 

Confusion Matrix: 
[[ 20   0   0   0  95]
 [  3   0   0   0  23]
 [  3   0   0   0  22]
 [  3   0   0   0  41]
 [  1   0   0   0 247]]


/opt/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/

## Task 8: Train models with other columns

## Based on the above results, the Naive Bayes model trained on unigram features is the best in terms of accuracy and overall F1 score

### Training the Naive Bayes with unigram features on title column

In [54]:
# use review for model
train_text_title = chatgpt_train["title"]
test_text_title = chatgpt_test["title"]

# set the n-gram range
vectorizer = CountVectorizer(ngram_range = (1,1))

# create training data representation
train_data_title_repr = vectorizer.fit_transform(train_text_title.values.astype('U'))
print("NUMBER OF FEATURES IN TRAINING DATASET WITH UNIGRAMS ON TITLE COLUMN")
print(train_data_title_repr.shape,"\n") 

# create test data representation
test_data_title_repr = vectorizer.transform(test_text_title.values.astype('U'))
print("NUMBER OF FEATURES IN TEST DATASET WITH UNIGRAMS ON TITLE COLUMN")
print(test_data_title_repr.shape,"\n") 

NUMBER OF FEATURES IN TRAINING DATASET WITH UNIGRAMS ON TITLE COLUMN
(1829, 1435) 

NUMBER OF FEATURES IN TEST DATASET WITH UNIGRAMS ON TITLE COLUMN
(458, 1435) 



In [55]:
# define true labels from train set
X_train = train_data_title_repr
y_train = chatgpt_train["rating"]
X_test = test_data_title_repr
y_test = chatgpt_test["rating"]

In [56]:
# Naive Bayes model
model_unigram_title = MultinomialNB()
model_unigram_title.fit(X_train, y_train)

MultinomialNB()

In [57]:
predictions = model_unigram_title.predict(X_test)
predictions

array([1, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 5, 5, 1, 5, 5, 5, 1, 5, 5, 5, 5,
       1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 1, 1, 5, 5,
       1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 5, 5, 1, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 3, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 4, 5, 5,
       5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5,
       5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 1,
       1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 1, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,

In [58]:
print ("Accuracy score: ", accuracy_score(y_test, predictions))
print ("Overall Recall score: ", recall_score(y_test, predictions, average='macro'))
print ("Overall Precision score: ", precision_score(y_test, predictions, average='macro'))
print ("Overall F1 score: ", f1_score(y_test, predictions, average='macro'), "\n")
print ("Individual label performance: ")
print (classification_report(y_test, predictions), "\n")
print ("Confusion Matrix: ")
print (confusion_matrix(y_test, predictions))

Accuracy score:  0.6135371179039302
Overall Recall score:  0.2935476220833865
Overall Precision score:  0.34369109499057526
Overall F1 score:  0.29203395065226506 

Individual label performance: 
              precision    recall  f1-score   support

           1       0.66      0.35      0.45       115
           2       0.00      0.00      0.00        26
           3       0.14      0.04      0.06        25
           4       0.29      0.14      0.18        44
           5       0.63      0.94      0.76       248

    accuracy                           0.61       458
   macro avg       0.34      0.29      0.29       458
weighted avg       0.54      0.61      0.55       458
 

Confusion Matrix: 
[[ 40   0   1   6  68]
 [  6   0   3   2  15]
 [  5   0   1   2  17]
 [  3   0   0   6  35]
 [  7   0   2   5 234]]


/opt/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/

## Training the Naive Bayes with unigram features on title + review column

#### Combine the title and review column for training and test data

In [59]:
chatgpt_train['title_review'] = chatgpt_train['title'] + " " + chatgpt_train['review']
chatgpt_test['title_review'] = chatgpt_test['title'] + " " + chatgpt_test['review']

#### Training

In [60]:
# use review for model
train_text_title_review = chatgpt_train['title_review']
test_text_title_review = chatgpt_test['title_review']

# set the n-gram range
vectorizer = CountVectorizer(ngram_range = (1,1))

# create training data representation
train_data_title_review_repr = vectorizer.fit_transform(train_text_title_review.values.astype('U'))
print("NUMBER OF FEATURES IN TRAINING DATASET WITH UNIGRAMS ON TITLE + REVIEW COLUMN")
print(train_data_title_review_repr.shape,"\n") 

# create test data representation
test_data_title_review_repr = vectorizer.transform(test_text_title_review.values.astype('U'))
print("NUMBER OF FEATURES IN TEST DATASET WITH UNIGRAMS ON TITLE + REVIEW COLUMN")
print(test_data_title_review_repr.shape,"\n") 

NUMBER OF FEATURES IN TRAINING DATASET WITH UNIGRAMS ON TITLE + REVIEW COLUMN
(1829, 5742) 

NUMBER OF FEATURES IN TEST DATASET WITH UNIGRAMS ON TITLE + REVIEW COLUMN
(458, 5742) 



In [61]:
# define true labels from train set

X_train = train_data_title_review_repr
y_train = chatgpt_train["rating"]
X_test = test_data_title_review_repr
y_test = chatgpt_test["rating"]

In [62]:
# Naive Bayes model
model_unigram_title_review = MultinomialNB()
model_unigram_title_review.fit(X_train, y_train)

MultinomialNB()

In [63]:
predictions = model_unigram_title_review.predict(X_test)
predictions

array([1, 1, 5, 3, 5, 5, 5, 5, 5, 1, 5, 3, 5, 1, 5, 5, 5, 1, 1, 5, 5, 5,
       1, 1, 3, 1, 5, 5, 1, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 1, 1, 5, 5,
       1, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 4, 5, 5, 1, 5, 5, 3, 1, 5,
       5, 5, 5, 5, 5, 3, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 5, 5,
       5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 1, 1, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 1,
       1, 1, 1, 5, 1, 1, 5, 5, 5, 5, 5, 2, 2, 5, 3, 3, 5, 5, 5, 5, 5, 5,
       5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 4, 5, 5, 2, 1, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,
       5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 3, 3, 5,
       5, 5, 5, 5, 5, 5, 4, 5, 2, 5, 2, 1, 2, 5, 5, 5, 1, 1, 5, 5, 5, 5,
       5, 5, 4, 5, 5, 5, 1, 3, 1, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5,
       1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 4, 5, 5, 5, 5,
       1, 4, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 1, 5,

In [64]:
print ("Accuracy score: ", accuracy_score(y_test, predictions))
print ("Overall Recall score: ", recall_score(y_test, predictions, average='macro'))
print ("Overall Precision score: ", precision_score(y_test, predictions, average='macro'))
print ("Overall F1 score: ", f1_score(y_test, predictions, average='macro'), "\n")
print ("Individual label performance: ")
print (classification_report(y_test, predictions), "\n")
print ("Confusion Matrix: ")
print (confusion_matrix(y_test, predictions))

Accuracy score:  0.6419213973799127
Overall Recall score:  0.3558083052991889
Overall Precision score:  0.4300508552935737
Overall F1 score:  0.3694878450223124 

Individual label performance: 
              precision    recall  f1-score   support

           1       0.58      0.50      0.53       115
           2       0.29      0.08      0.12        26
           3       0.09      0.08      0.09        25
           4       0.48      0.23      0.31        44
           5       0.72      0.90      0.80       248

    accuracy                           0.64       458
   macro avg       0.43      0.36      0.37       458
weighted avg       0.60      0.64      0.61       458
 

Confusion Matrix: 
[[ 57   2   6   4  46]
 [ 10   2   5   2   7]
 [ 11   0   2   2  10]
 [  7   0   4  10  23]
 [ 14   3   5   3 223]]


## Task 10: Error Analysis

In [65]:
prediction = predictions
labels = chatgpt_test["rating"]
inputs = chatgpt_test["title"] + " --- " + chatgpt_test["review"]

unique_rating = chatgpt_test["rating"].unique()
unique_rating.sort()

for rating in unique_rating:
    count = 0
    print(f"RATING {rating!r}")
    print("----------")
    
    for i in range(len(inputs)):
        if prediction[i] != labels[i] and labels[i] == rating and count < 3:
            count += 1
            print("Title --- Review: ", inputs[i])
            print(f'It has been classified with rating {prediction[i]!r} but has actual rating {labels[i]!r}')
            print ("\n")

RATING 1
----------
Title --- Review:  your gpt 4 is fake --- Fix it
It has been classified with rating 5 but has actual rating 1


Title --- Review:  Version 3 --- Old version of ChatGPT.
It has been classified with rating 5 but has actual rating 1


Title --- Review:  Unusable on iPad --- Unusable on iPad Pro currently.
It has been classified with rating 3 but has actual rating 1


RATING 2
----------
Title --- Review:  error unsupported country --- cant login
It has been classified with rating 1 but has actual rating 2


Title --- Review:  no gpt4.0 ???cheater --- i subscribe the plus for gpt4???but just get gpt3.0 with fake label 4.0.release real gpt4 now?????????
It has been classified with rating 1 but has actual rating 2


Title --- Review:  Outdated --- This app is based on chatgpt 3 (cutoff date Sep 2021)
It has been classified with rating 3 but has actual rating 2


RATING 3
----------
Title --- Review:  Why not support ios15? --- ????
It has been classified with rating 1 but

## Task 11: Three-way vs five-way classification

### Creating a new column for 3 way classes -> positive, negative and neutral

In [66]:
def three_way_classes(rating):
    if rating == 1 or rating == 2:
        val = 'negative'
    elif rating == 4 or rating == 5:
        val = 'positive'
    else:
        val = 'neutral'
        
    return val

#### Apply it to the train and test dataset

In [67]:
chatgpt_train['3_way_classes'] = chatgpt_train['rating'].apply(three_way_classes)
chatgpt_train[['3_way_classes']]

,3_way_classes
0,positive
1,positive
2,positive
3,positive
4,positive
...,...
1829,negative
1830,positive
1831,negative
1832,negative


In [68]:
chatgpt_test['3_way_classes'] = chatgpt_test['rating'].apply(three_way_classes)
chatgpt_test[['3_way_classes']]

,3_way_classes
0,negative
1,negative
2,negative
3,positive
4,positive
...,...
453,positive
454,positive
455,positive
456,negative


## Naive Bayes model trained on unigram features is the best in terms of accuracy and overall F1 score

### Training the Naive Bayes with unigram features on the 3 way classification

In [69]:
# use review for model
train_text_title_review = chatgpt_train['review']
test_text_title_review = chatgpt_test['review']

# set the n-gram range
vectorizer = CountVectorizer(ngram_range = (1,1))

# create training data representation
train_data_title_review_repr = vectorizer.fit_transform(train_text_title_review.values.astype('U'))
print("NUMBER OF FEATURES IN TRAINING DATASET WITH 3 WAY CLASSICATION")
print(train_data_title_review_repr.shape,"\n") 

# create test data representation
test_data_title_review_repr = vectorizer.transform(test_text_title_review.values.astype('U'))
print("NUMBER OF FEATURES IN TEST DATASET WITH 3 WAY CLASSICATION")
print(test_data_title_review_repr.shape,"\n") 

NUMBER OF FEATURES IN TRAINING DATASET WITH 3 WAY CLASSICATION
(1829, 5551) 

NUMBER OF FEATURES IN TEST DATASET WITH 3 WAY CLASSICATION
(458, 5551) 



In [70]:
# define true labels from train set

X_train = train_data_title_review_repr
y_train = chatgpt_train["3_way_classes"]
X_test = test_data_title_review_repr
y_test = chatgpt_test["3_way_classes"]

In [71]:
# Naive Bayes model

model_3_way_classification = MultinomialNB()
model_3_way_classification.fit(X_train, y_train)

MultinomialNB()

In [72]:
predictions = model_3_way_classification.predict(X_test)
predictions

array(['negative', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'negative', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'negative', 'positive', 'positive', 'negative', 'positive',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'positive', 'positive',
       'negative', 'neutral', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'negative',
       'positive', 'neutral', 'neutral', 'positive', 'positive',
       'negative', 'positive', 'positive', 'negative', 'negative',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'positive', 'positive', 'negative', 'positive'

In [73]:
print ("Accuracy score: ", accuracy_score(y_test, predictions))
print ("Overall Recall score: ", recall_score(y_test, predictions, average='macro'))
print ("Overall Precision score: ", precision_score(y_test, predictions, average='macro'))
print ("Overall F1 score: ", f1_score(y_test, predictions, average='macro'), "\n")
print ("Individual label performance: ")
print (classification_report(y_test, predictions), "\n")
print ("Confusion Matrix: ")
print (confusion_matrix(y_test, predictions))

Accuracy score:  0.7358078602620087
Overall Recall score:  0.5007752841737103
Overall Precision score:  0.5420844714259355
Overall F1 score:  0.5116057233704292 

Individual label performance: 
              precision    recall  f1-score   support

    negative       0.69      0.53      0.60       141
     neutral       0.17      0.08      0.11        25
    positive       0.77      0.89      0.83       292

    accuracy                           0.74       458
   macro avg       0.54      0.50      0.51       458
weighted avg       0.71      0.74      0.72       458
 

Confusion Matrix: 
[[ 75   4  62]
 [  8   2  15]
 [ 26   6 260]]


In [74]:
chatgpt_train['rating'].value_counts()

5    885
1    379
4    258
3    195
2    112
Name: rating, dtype: int64